In [1]:
from sktime.datasets import load_acsf1

X_train_uni, y_train_uni = load_acsf1(split='train') # Load the training instances  of acsf1
X_train_uni.head()

ModuleNotFoundError: No module named 'sktime'

In [3]:
from sktime.datasets import load_basic_motions

X_train_multi, y_train_multi = load_basic_motions(split='train') # Load the training instances of basic motion
X_train_multi.head()

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5
0,0 0.079106 1 0.079106 2 -0.903497 3...,0 0.394032 1 0.394032 2 -3.666397 3...,0 0.551444 1 0.551444 2 -0.282844 3...,0 0.351565 1 0.351565 2 -0.095881 3...,0 0.023970 1 0.023970 2 -0.319605 3...,0 0.633883 1 0.633883 2 0.972131 3...
1,0 0.377751 1 0.377751 2 2.952965 3...,0 -0.610850 1 -0.610850 2 0.970717 3...,0 -0.147376 1 -0.147376 2 -5.962515 3...,0 -0.103872 1 -0.103872 2 -7.593275 3...,0 -0.109198 1 -0.109198 2 -0.697804 3...,0 -0.037287 1 -0.037287 2 -2.865789 3...
2,0 -0.813905 1 -0.813905 2 -0.424628 3...,0 0.825666 1 0.825666 2 -1.305033 3...,0 0.032712 1 0.032712 2 0.826170 3...,0 0.021307 1 0.021307 2 -0.372872 3...,0 0.122515 1 0.122515 2 -0.045277 3...,0 0.775041 1 0.775041 2 0.383526 3...
3,0 0.289855 1 0.289855 2 -0.669185 3...,0 0.284130 1 0.284130 2 -0.210466 3...,0 0.213680 1 0.213680 2 0.252267 3...,0 -0.314278 1 -0.314278 2 0.018644 3...,0 0.074574 1 0.074574 2 0.007990 3...,0 -0.079901 1 -0.079901 2 0.237040 3...
4,0 -0.123238 1 -0.123238 2 -0.249547 3...,0 0.379341 1 0.379341 2 0.541501 3...,0 -0.286006 1 -0.286006 2 0.208420 3...,0 -0.098545 1 -0.098545 2 -0.023970 3...,0 0.058594 1 0.058594 2 0.175783 3...,0 -0.074574 1 -0.074574 2 0.114525 3...


In [4]:
from sktime.datatypes import convert_to
import warnings
warnings.filterwarnings('ignore') # Hide warnings that can generate and clutter notebook

X_train_uni_np = convert_to(X_train_uni, 'numpy3D') # Convert dataframe to numpy
X_train_multi_np = convert_to(X_train_multi, 'numpy3D') # Convert dataframe to numpy

n_uni, d_uni, m_uni = X_train_uni_np.shape # The original shape is n, d, m
X_train_uni_d_m = X_train_uni_np.copy() # Used for libraries (sktime) that want d, m
X_train_uni_m_d = X_train_uni_np.reshape((n_uni, m_uni, d_uni)) # Used for libraries that want m, d

n_multi, d_multi, m_multi = X_train_multi_np.shape # The original shape is n, d, m
X_train_multi_d_m = X_train_multi_np.copy() # Used for libraries (sktime) that want d, m
X_train_multi_m_d = X_train_multi_np.reshape((n_multi, m_multi, d_multi)) # User for libraries that want m, d

In [17]:
import time
from dtw import DTW as dtw_python_DTW

def timing_experiment(X, distance_callable, distance_params = None):
    """Time the average time it takes to take the distance from the first time series
    to all of the other time series in X.

    Parameters
    ----------
    X: np.ndarray
        A dataset of time series.
    distance_callable: Callable
        A callable that is the distance function to time.

    Returns
    -------
    float
        Average time it took to run a distance
    list[float]
        List of floats containing the distances.
    """
    if distance_params is None:
        distance_params = {}
    first_val = X[0]
    total_time = 0
    distances = []
    for val in X:
        start = time.time()
        curr_dist = distance_callable(first_val, val, **distance_params)
        total_time += time.time() - start
        if isinstance(curr_dist, dtw_python_DTW):
            curr_dist = curr_dist.distance
        distances.append(curr_dist)

    return total_time / len(X), distances

In [18]:
from sktime.distances import dtw_distance as sktime_dtw
sktime_uni_time, sktime_uni_distances = timing_experiment(X_train_uni_d_m, sktime_dtw)
sktime_multi_time, sktime_multi_distances = timing_experiment(X_train_multi_d_m, sktime_dtw)
print(sktime_uni_time)
print(sktime_multi_time)

0.14548295974731446
0.006346142292022705


In [19]:
from tslearn.metrics import dtw as tslearn_dtw
tslearn_uni_time, tslearn_uni_distances = timing_experiment(X_train_uni_m_d, tslearn_dtw)
tslearn_multi_time, tslearn_multi_distances = timing_experiment(X_train_multi_m_d, tslearn_dtw)
print(tslearn_uni_time)
print(tslearn_multi_time)

0.01870164155960083
0.00010004639625549316


In [20]:
from dtw import dtw as dtw_python_dtw
dtw_python_uni_time, dtw_python_uni_distances = timing_experiment(X_train_uni_d_m, dtw_python_dtw)
dtw_python_multi_time, dtw_python_multi_distances = timing_experiment(X_train_uni_d_m, dtw_python_dtw)
print(dtw_python_uni_time)
print(dtw_python_multi_time)

0.07066751718521118
0.0701207971572876


In [25]:
from rust_dtw import dtw as rust_dtw_dtw
dtw_c_uni_time, dtw_dtai_uni_distances = timing_experiment(X_train_uni_d_m, rust_dtw_dtw, distance_params={"window": X_train_uni_d_m.shape[1], "distance_mode": "euclidean"})
# dtw_c_multi_time, dtw_dtai_multi_distances = timing_experiment(X_train_uni_d_m, rust_dtw_dtw, distance_params={"window": X_train_uni_d_m.shape[1], "distance_mode": "euclidean"})
print(dtw_c_uni_time)
# print(dtw_c_multi_time)

TypeError: argument 's': Shape Mismatch:
 from=(dim=2, dtype=Float64), to=(dim=1, dtype=Float64)